In [1]:
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, get_scheduler
from datasets import load_dataset,Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from torch.optim import AdamW
import pandas as pd

/root/miniconda3/envs/countVctorRidgeclassification/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
# 1. 加载测试集
df_test = pd.read_csv('/root/data/test_a.csv', sep='\t')

In [5]:
test_dataset = Dataset.from_pandas(df_test)
test_dataset

Dataset({
    features: ['text'],
    num_rows: 50000
})

In [6]:
# 2. 数据预处理
tokenizer = BertTokenizer.from_pretrained('/root/model/bert-base-chinese')

In [7]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True, max_length=128)

In [8]:
encoded_dataset = test_dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 50000/50000 [13:36<00:00, 61.27 examples/s]


In [9]:
test_dataset_torch = encoded_dataset.with_format('torch')

In [41]:
test_dataset_torch

Dataset({
    features: ['text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 50000
})

In [42]:
test_loader = DataLoader(test_dataset_torch, batch_size=16)

In [43]:
test_loader

In [29]:
# 3. 加载模型
# 3.1  初始化模型（架构必须与保存时相同）
model = BertForSequenceClassification.from_pretrained('/root/model/bert-base-chinese', num_labels=14)  # 假设有2个分类标签

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /root/model/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
# 3.2 加载状态字典
model.load_state_dict(torch.load('BertForSequenceClassification.pth'))

<All keys matched successfully>

In [31]:
 # 3.3 将模型设置为评估模式（如果需要）
model.eval()
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [47]:
# 4. 测试集评估
all_preds = []
with torch.no_grad():
    for batch in test_loader:
        # print(batch)
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
    
         # 前向传播获取预测结果
        outputs = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        # print(preds)
        all_preds.extend(preds)

In [49]:
len(all_preds)

50000

In [51]:
# 5. 结果存储
df_test['label'] = all_preds
df_test['label'].to_csv("/root/data/BertForSequenceClassification .csv", index=False) 